In [4]:
import numpy as np
import pandas as pd
import re
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer

In [5]:
# Download required datasets
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
#Reading DataSet as DataFrame
df = pd.read_csv(r"C:\Users\DELL\Desktop\Fake news detection\train.csv")

In [7]:
df.shape

(20800, 5)

In [8]:
# print the first 5 rows
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
# Check for missing values
print(df.isnull().sum())

id           0
title      558
author    1957
text        39
label        0
dtype: int64


In [11]:
# replacing the null values with empty string
df = df.fillna('')

In [12]:
# concatenates the author name and news title
df['content'] = df['author']+' '+df['title']

In [14]:
print(df['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [15]:
# Split the data into features (X) and labels (y)
X = df.drop(columns='label', axis=1)
Y = df['label']

In [23]:

# Initialize the stemmer
port_stem = PorterStemmer()

def stemming(content):
    # Remove non-alphabetic characters
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    
    # Convert to lowercase
    stemmed_content = stemmed_content.lower()
    
    # Tokenization
    stemmed_content = stemmed_content.split()
    
    # Stemming and Stopwords Removal
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if word not in stopwords.words('english')]
    
    # Join words back into a single string
    return ' '.join(stemmed_content)

In [26]:
df['content'] = df['content'].apply(stemming)

In [27]:
# separates the dataset into features (X) and labels (Y) 
X = df['content'].values
Y = df['label'].values

In [28]:
# Initialize the TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
vectorizer.fit(X)
X = vectorizer.transform(X)

In [29]:
print(X)

  (0, 15575)	0.3036992657412891
  (0, 8847)	0.3876556342629956
  (0, 8568)	0.31145512661943237
  (0, 7634)	0.2642526302631701
  (0, 6951)	0.2332158773846461
  (0, 3766)	0.28843449347042016
  (0, 3575)	0.3837081797328416
  (0, 2936)	0.2631788024287867
  (0, 2461)	0.3919795814760503
  (0, 266)	0.28797390973856846
  (1, 16673)	0.30071745655510157
  (1, 6762)	0.1904660198296849
  (1, 5459)	0.7143299355715573
  (1, 3543)	0.26373768806048464
  (1, 2791)	0.19094574062359204
  (1, 2203)	0.3827320386859759
  (1, 1875)	0.15521974226349364
  (1, 1480)	0.2939891562094648
  (2, 15500)	0.5958062690640037
  (2, 3079)	0.6610951489703274
  (2, 2920)	0.4560351891597104
  (3, 13796)	0.3643396661700562
  (3, 11939)	0.49295210164456477
  (3, 8078)	0.2481173591314576
  (3, 7698)	0.4262786599469829
  :	:
  (20797, 15186)	0.0834821390306957
  (20797, 14866)	0.31881045786261725
  (20797, 13031)	0.25400168580178045
  (20797, 12253)	0.27894826579153137
  (20797, 12049)	0.2535207421929807
  (20797, 10232)	0.08224

In [30]:
# Splitting the dataset into training and test data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

In [31]:
# Training the Logistic Regression Model:
model = LogisticRegression()

In [32]:
model.fit(X_train, Y_train)

LogisticRegression()

In [33]:
# accuracy score of the training data
Train_prediction = model.predict(X_train)
training_accuracy = accuracy_score(Train_prediction, Y_train)

In [34]:
print('Accuracy score of the training data : ', training_accuracy)

Accuracy score of the training data :  0.9870192307692308


In [35]:
# accuracy score of the test data
Test_prediction = model.predict(X_test)
test_accuracy = accuracy_score(Test_prediction, Y_test)

In [44]:
print('Accuracy score of the test data : ', training_accuracy)

Accuracy score of the test data :  0.9870192307692308


In [58]:
X_new = X_test[233]
prediction = model.predict(X_new)
print(prediction)
if (prediction[0]==0):
    print('Real')
else:
    print('Fake')
    
print(Y_test[233])

[0]
Real
0


In [42]:
import joblib

# Save the model
joblib.dump(model, r'C:\Users\DELL\Desktop\Fake news detection\fake_news_detector.pkl')

# Save the vectorizer
joblib.dump(vectorizer, r'C:\Users\DELL\Desktop\Fake news detection\tfidf_vectorizer.pkl')

['C:\\Users\\DELL\\Desktop\\Fake news detection\\tfidf_vectorizer.pkl']